## Introduccion

El siguiente proyectoi es un recomendador d peliculas sencillo, basado en MovieLens con 100K de datos para entreno

# Primeros pasos

Importación e las librerias básicas de Panas y N

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import ipywidgets as widgets
from IPython.display import display
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
import warnings
warnings.filterwarnings("ignore")

## Carga de Datos

Se hace la carga de los datasets descargados desde Kaggle


In [3]:
df_ratings = pd.read_csv("ml-100k/u.data", sep="\t", names=["user_id", "movie_id", "rating", "timestamp"])
df_movies = pd.read_csv("ml-100k/u.item", sep="|", encoding="latin-1", header=None, usecols=[0,1], names=["movie_id","title"])
df = pd.merge(df_ratings, df_movies, on="movie_id")
df["liked"] = (df["rating"] >= 4).astype(int)

## Crear la matriz de usuarios/películas

(pivot_table) reorganiza el Dataframe original (dF) para crear una nueva tabla 


In [4]:
titles = sorted(df["title"].unique())
title_to_id = df.drop_duplicates(subset=["movie_id"]).set_index("title")["movie_id"].to_dict()
scaler = StandardScaler()
clf = RandomForestClassifier()
X = df[["user_id","movie_id"]]
y = df["liked"]
X_scaled = scaler.fit_transform(X)
clf.fit(X_scaled, y)

RandomForestClassifier()

## configuramos el modelo
con RandomForestClassifier y entrenamos el modelo inicial 

In [ ]:
#  Preparar datos y modelo inicial
titles = sorted(df["title"].unique())
title_to_id = df.drop_duplicates(subset=["movie_id"]).set_index("title")["movie_id"].to_dict()
scaler = StandardScaler()
clf = RandomForestClassifier()
X = df[["user_id","movie_id"]]
y = df["liked"]
X_scaled = scaler.fit_transform(X)
clf.fit(X_scaled, y)

 ## recomendaciones por correlación
 se genera la funcion que correlaciones los datos del codigo 

In [ ]:
## Función de recomendación por correlación
def get_similar(movie, min_count=50, min_mean=4):
    mat = df.pivot_table(index="user_id", columns="title", values="rating")
    counts = df.groupby("title")["rating"].count()
    corr = mat.corrwith(mat[movie])
    rec = pd.DataFrame(corr, columns=["corr"]).join(counts.rename("count")).dropna()
    mean_r = df.groupby("title")["rating"].mean()
    rec = rec.join(mean_r.rename("mean"))
    return rec[(rec["count"]>=min_count)&(rec["mean"]>=min_mean)].sort_values("corr",ascending=False)

## Widgets para recomendaciones basadas en correlación

Configurar widgets y eventos en UNICA CELDA

In [5]:
selector = widgets.Dropdown(options=titles, description="Película:")
slider = widgets.IntSlider(value=4, min=1, max=5, description="Min Rating:")
btn_rec = widgets.Button(description="Ver Similares")
out_rec = widgets.Output()

movie_sel = widgets.Dropdown(options=titles, description="Tu Película:")
rate_sel = widgets.IntSlider(value=4, min=1, max=5, description="Tu Rating:")
btn_rate = widgets.Button(description="Enviar Rating")
out_rate = widgets.Output()

## Generamos la "interfaz" del ususario 

In [6]:
# --- Binding de eventos ---
def show_similares(b):
    with out_rec:
        out_rec.clear_output()
        res = get_similar(selector.value, min_count=50, min_mean=slider.value).head(10)
        display(f"🎬 Similares a '{selector.value}' (>= {slider.value}):")
        display(res)
btn_rec.on_click(show_similares)

def add_rating(b):
    with out_rate:
        out_rate.clear_output()
        uid_new = df["user_id"].max() + 1
        mid = title_to_id[movie_sel.value]
        ts = int(pd.Timestamp.now().timestamp())
        # Insertar fila: user_id, movie_id, rating, timestamp, title, liked
        df.loc[len(df)] = [uid_new, mid, rate_sel.value, ts, movie_sel.value, int(rate_sel.value >= 4)]
        # Reentrenar
        Xn = df[["user_id","movie_id"]]
        yn = df["liked"]
        Xs = scaler.fit_transform(Xn)
        clf.fit(Xs, yn)
        display(f"❤️ Rating agregado: '{movie_sel.value}' -> {rate_sel.value}")
btn_rate.on_click(add_rating)

# --- Mostrar interfaz ---
display(widgets.VBox([
    selector, slider, btn_rec, out_rec,
    movie_sel, rate_sel, btn_rate, out_rate
]))